In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils

import altair as alt
import pandas as pd
import numpy as np
from constants import COLUMNS
from utils import (
    read_combined_by_site_demographics_df,
    read_lab_drop_adjusted_df,
    read_lab_drop_unadjusted_df,
    get_siteid_anonymous_map,
    get_anonymousid_color_maps,
    apply_theme
)
from web import for_website

In [ ]:
# Titles
NUM_SITES = len(read_combined_by_site_demographics_df()[COLUMNS.SITE_ID].unique())
DATA_DATE = "2020-04-11"
VIS_DATE = "2020-04-13"
SUBTITLE = f"Data as of {DATA_DATE} | {NUM_SITES} Sites | Plots generated on {VIS_DATE}"

In [ ]:
def process_lab_drop(df):

    consistent_loinc = {
        "alanine aminotransferase (ALT)": "Alanine aminotransferase (U/L)",
        "albumin": "Albumin (g/dL)",
        "aspartate aminotransferase (AST)": "Aspartate aminotransferase (U/L)",
        "total bilirubin": "Total bilirubin (mg/dL)",
        "C-reactive protein (CRP)": "C-reactive protein (mg/dL)",
        "creatinine": "Creatinine (mg/dL)",
        "lactate dehydrogenase (LDH)": "Lactate dehydrogenase (U/L)",
        "cardiac troponin": "Cardiac troponin (ng/mL)",
        "prothrombin time (PT)": "Prothrombin time (s)",
        "white blood cell count (Leukocytes)": "White blood cell count (10*3/uL)",
        "lymphocyte count": "Lymphocyte count (10*3/uL)",
        "neutrophil count": "Neutrophil count (10*3/uL)",
        "D-dimer": "D-dimer",
        "procalcitonin": "Procalcitonin (ng/mL)",
    }
    df["Lab"] = df["Lab"].apply(lambda x: consistent_loinc[x])

    # Wide to long
    df = pd.melt(df, id_vars=(["Lab", COLUMNS.DAYS_SINCE_POSITIVE]))
    df = df.rename(columns={"variable": COLUMNS.SITE_ID, "value": "percentage"})

    return df

unadjusted_df = read_lab_drop_unadjusted_df()
unadjusted_df = process_lab_drop(unadjusted_df)

adjusted_df = read_lab_drop_adjusted_df()
adjusted_df = process_lab_drop(adjusted_df)

In [ ]:
alt.data_transformers.disable_max_rows() # Use rows more than 5000

In [ ]:
def lab_drop_plot(df, title):

    unique_labs = df["Lab"].unique().tolist()
    lab_dropdown = alt.binding_select(options=unique_labs)
    lab_selection = alt.selection_single(fields=["Lab"], bind=lab_dropdown, init={"Lab": unique_labs[0]}, name="Lab")
    legend_selection = alt.selection_multi(fields=[COLUMNS.SITE_ID], bind="legend")

    ANONYMOUS_SITES = list(get_anonymousid_color_maps()[1].keys())
    ANONYMOUS_COLORS = list(get_anonymousid_color_maps()[1].values())

    line = alt.Chart(df).mark_line().encode(
        x=alt.X(
            COLUMNS.DAYS_SINCE_POSITIVE, 
            scale=alt.Scale(
                domain=[-1, 28], 
                nice=False, 
                clamp=False), 
            title="Days since positive"
        ),
        y=alt.Y(
            "percentage", 
            axis=alt.Axis(format="%"),
            scale=alt.Scale(domain=[-0.05, 1.05], nice=False, clamp=False), 
            title="Percentage of measured (%)"
        ),
        color=alt.Color(COLUMNS.SITE_ID, scale=alt.Scale(domain=ANONYMOUS_SITES, range=ANONYMOUS_COLORS), title=None),
        opacity=alt.value(0.5)
    )

    circle = line.mark_circle(size=30)

    plot = (
        line 
        + 
        circle
    ).transform_filter(
        lab_selection
    ).transform_filter(
        legend_selection
    ).add_selection(
        legend_selection
    ).add_selection(
        lab_selection
    ).properties(
        title={
            "text": title,
            "dx": 70,
            "subtitle": SUBTITLE,
            "subtitleColor": "gray",
        },
        width=500, height=400
    ).interactive()

    return apply_theme(plot, legend_orient="right")

In [ ]:
unadjusted_plot = lab_drop_plot(unadjusted_df, "Percentage of Measured Relative to Baseline")

for_website(unadjusted_plot, "Labs", "Percentage of measured relative to baseline")

unadjusted_plot

In [ ]:
adjusted_plot = lab_drop_plot(adjusted_df, title="Percentage of Measured Among at Risk")

for_website(adjusted_plot, "Labs", "Percentage of measured among at risk") # or "Baseline"

adjusted_plot